<a href="https://colab.research.google.com/github/bintangnabiil/Hands-On-Machine-Learning-with-Scikit-Learn-Keras-and-TensorFlow/blob/main/Rangkuman_Chapter_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Chapter 12: Custom Models and Training with TensorFlow
Chapter ini membahas cara membuat model kustom dan proses training menggunakan TensorFlow tingkat rendah. Kita akan mempelajari cara kerja internal TensorFlow, membuat custom layers, custom training loops, dan menggunakan TensorFlow's automatic differentiation untuk optimasi yang lebih fleksibel.

##1.TensorFlow's Lower-Level API
TensorFlow menyediakan API tingkat rendah yang memberikan kontrol penuh atas proses training dan model architecture. Berbeda dengan Keras yang high-level, API ini memungkinkan kustomisasi yang sangat detail.

###Keuntungan menggunakan Lower-Level API:

- Kontrol penuh atas forward pass dan backward pass
- Kemampuan untuk mengimplementasikan algoritma training kustom
- Debugging yang lebih mendalam
- Optimasi performa untuk kasus khusus

##2. Tensors dan Operations
Tensor adalah struktur data dasar di TensorFlow, mirip dengan NumPy arrays tetapi dapat dijalankan di GPU dan mendukung automatic differentiation.

###Karakteristik Tensor:

- Immutable (tidak dapat diubah setelah dibuat)
- Memiliki dtype (tipe data) dan shape
- Dapat dibuat dari konstanta, variabel, atau operasi lain

##3. Variables dan Constants
Variables digunakan untuk menyimpan state yang dapat berubah (seperti weights), sedangkan Constants untuk nilai yang tetap.

###Perbedaan utama:

- Variables: Mutable, dapat diubah dengan .assign()
- Constants: Immutable, nilainya tetap

##4. Custom Layers
Membuat layer kustom memungkinkan implementasi operasi yang tidak tersedia di Keras standard library.

###Cara membuat custom layer:

- Inherit dari tf.keras.layers.Layer
- Override method build() untuk inisialisasi weights
- Override method call() untuk forward pass
- Optionally override get_config() untuk serialization

##5. Custom Models
Selain custom layers, kita juga dapat membuat model yang sepenuhnya kustom dengan logic yang kompleks.

###Pendekatan untuk custom models:

- Subclassing tf.keras.Model
- Menggunakan Functional API dengan custom layers
- Membuat model dari scratch tanpa Keras

##6. Automatic Differentiation
TensorFlow menggunakan automatic differentiation (AutoDiff) untuk menghitung gradients secara otomatis.

###GradientTape:

- Merekam operasi untuk forward pass
- Menghitung gradients untuk backward pass
- Mendukung higher-order derivatives

##7. Custom Training Loops
Training loop kustom memberikan kontrol penuh atas proses training, termasuk bagaimana loss dihitung dan bagaimana weights diupdate.

###Komponen custom training loop:

- Forward pass manual
- Loss computation
- Gradient computation dengan GradientTape
- Weight updates dengan optimizer

##8. Custom Metrics dan Losses
Implementasi metrics dan loss functions kustom untuk kasus-kasus khusus yang tidak tersedia di TensorFlow standard library.

##9. Performance Optimization
Teknik-teknik untuk mengoptimalkan performa model kustom, termasuk penggunaan @tf.function decorator dan graph optimization.

#Implementasi Kode
##1. Dasar-dasar Tensors dan Operations

In [1]:
import tensorflow as tf
import numpy as np

# Membuat tensors
t1 = tf.constant([[1., 2., 3.], [4., 5., 6.]])
t2 = tf.constant([[7., 8.], [9., 10.], [11., 12.]])

print("Tensor 1 shape:", t1.shape)
print("Tensor 1 dtype:", t1.dtype)

# Operasi tensor
result = tf.matmul(t1, t2)
print("Matrix multiplication result:")
print(result)

# Menggunakan Variables
w = tf.Variable([[1.0, 2.0], [3.0, 4.0]])
print("Initial weights:", w)

# Mengubah nilai variable
w.assign([[5.0, 6.0], [7.0, 8.0]])
print("Updated weights:", w)

Tensor 1 shape: (2, 3)
Tensor 1 dtype: <dtype: 'float32'>
Matrix multiplication result:
tf.Tensor(
[[ 58.  64.]
 [139. 154.]], shape=(2, 2), dtype=float32)
Initial weights: <tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>
Updated weights: <tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[5., 6.],
       [7., 8.]], dtype=float32)>


##2. Custom Layer Implementation

In [2]:
class CustomDenseLayer(tf.keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        # Inisialisasi weights
        self.kernel = self.add_weight(
            name='kernel',
            shape=(input_shape[-1], self.units),
            initializer='glorot_uniform',
            trainable=True
        )
        self.bias = self.add_weight(
            name='bias',
            shape=(self.units,),
            initializer='zeros',
            trainable=True
        )
        super().build(input_shape)

    def call(self, inputs):
        # Forward pass
        output = tf.matmul(inputs, self.kernel) + self.bias
        if self.activation:
            output = self.activation(output)
        return output

    def get_config(self):
        config = super().get_config()
        config.update({
            'units': self.units,
            'activation': tf.keras.activations.serialize(self.activation)
        })
        return config

# Menggunakan custom layer
model = tf.keras.Sequential([
    CustomDenseLayer(64, activation='relu'),
    CustomDenseLayer(32, activation='relu'),
    CustomDenseLayer(10, activation='softmax')
])

# Compile model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

##3. Custom Model dengan Subclassing

In [3]:
class CustomModel(tf.keras.Model):
    def __init__(self, num_classes=10):
        super().__init__()
        self.num_classes = num_classes

        # Define layers
        self.dense1 = tf.keras.layers.Dense(64, activation='relu')
        self.dropout1 = tf.keras.layers.Dropout(0.2)
        self.dense2 = tf.keras.layers.Dense(32, activation='relu')
        self.dropout2 = tf.keras.layers.Dropout(0.2)
        self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs, training=False):
        # Forward pass dengan logic kustom
        x = self.dense1(inputs)
        x = self.dropout1(x, training=training)
        x = self.dense2(x)
        x = self.dropout2(x, training=training)

        # Tambahkan skip connection sederhana
        if inputs.shape[-1] == x.shape[-1]:
            x = x + inputs

        return self.classifier(x)

# Inisialisasi model
custom_model = CustomModel(num_classes=10)

# Build model dengan contoh input
dummy_input = tf.random.normal((1, 784))  # Contoh untuk MNIST
_ = custom_model(dummy_input)

print("Model summary:")
custom_model.summary()

Model summary:


Model: "custom_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (1, 64)                │        50,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (1, 32)                │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (1, 10)                │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 52,650 (205.66 KB)

 Trainable params: 52,650 (205.66 KB)

 Non-trainable params: 0 (0.00 B)

##4. Automatic Differentiation dengan GradientTape

In [4]:
# Contoh sederhana automatic differentiation
def simple_function(x):
    return x**2 + 2*x + 1

x = tf.Variable(3.0)

# Menghitung gradient
with tf.GradientTape() as tape:
    y = simple_function(x)

dy_dx = tape.gradient(y, x)
print(f"f(x) = x^2 + 2x + 1")
print(f"f'(x) at x=3: {dy_dx}")

# Contoh dengan neural network
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
])

x_train = tf.random.normal((100, 5))
y_train = tf.random.normal((100, 1))

with tf.GradientTape() as tape:
    predictions = model(x_train)
    loss = tf.keras.losses.mse(y_train, predictions)

# Menghitung gradients
gradients = tape.gradient(loss, model.trainable_variables)
print("Gradients computed for", len(gradients), "variables")

f(x) = x^2 + 2x + 1
f'(x) at x=3: 8.0
Gradients computed for 4 variables


##5. Custom Training Loop

In [5]:
# Persiapan data (contoh dengan dataset sederhana)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 784).astype('float32') / 255.0
x_test = x_test.reshape(-1, 784).astype('float32') / 255.0

# Convert to tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.batch(32).shuffle(1000)

# Model dan optimizer
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

# Custom training step
@tf.function
def train_step(x_batch, y_batch):
    with tf.GradientTape() as tape:
        predictions = model(x_batch, training=True)
        loss = loss_fn(y_batch, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss

# Training loop
epochs = 3
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    epoch_loss = 0
    num_batches = 0

    for x_batch, y_batch in train_dataset:
        loss = train_step(x_batch, y_batch)
        epoch_loss += loss
        num_batches += 1

    avg_loss = epoch_loss / num_batches
    print(f"Average loss: {avg_loss:.4f}")

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/3
Average loss: 0.3001
Epoch 2/3
Average loss: 0.1450
Epoch 3/3
Average loss: 0.1089


##6. Custom Loss Function

In [6]:
class CustomLoss(tf.keras.losses.Loss):
    def __init__(self, alpha=1.0, name='custom_loss'):
        super().__init__(name=name)
        self.alpha = alpha

    def call(self, y_true, y_pred):
        # Custom loss yang menggabungkan MSE dan MAE
        mse = tf.keras.losses.mse(y_true, y_pred)
        mae = tf.keras.losses.mae(y_true, y_pred)
        return self.alpha * mse + (1 - self.alpha) * mae

# Menggunakan custom loss
custom_loss = CustomLoss(alpha=0.7)

# Contoh penggunaan
y_true = tf.constant([[1.0], [2.0], [3.0]])
y_pred = tf.constant([[1.1], [1.9], [3.2]])

loss_value = custom_loss(y_true, y_pred)
print(f"Custom loss value: {loss_value}")

Custom loss value: 0.054000020027160645


##7. Custom Metric

In [7]:
class CustomAccuracy(tf.keras.metrics.Metric):
    def __init__(self, name='custom_accuracy', **kwargs):
        super().__init__(name=name, **kwargs)
        self.total = self.add_weight(name='total', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Custom accuracy yang mempertimbangkan toleransi
        tolerance = 0.1
        matches = tf.abs(y_true - y_pred) <= tolerance
        matches = tf.cast(matches, self.dtype)

        if sample_weight is not None:
            sample_weight = tf.cast(sample_weight, self.dtype)
            matches = tf.multiply(matches, sample_weight)

        self.total.assign_add(tf.reduce_sum(matches))
        self.count.assign_add(tf.cast(tf.size(y_true), self.dtype))

    def result(self):
        return self.total / self.count

    def reset_state(self):
        self.total.assign(0)
        self.count.assign(0)

# Menggunakan custom metric
custom_metric = CustomAccuracy()

# Contoh penggunaan
y_true = tf.constant([1.0, 2.0, 3.0, 4.0])
y_pred = tf.constant([1.05, 1.95, 3.08, 4.12])

custom_metric.update_state(y_true, y_pred)
print(f"Custom accuracy: {custom_metric.result()}")

Custom accuracy: 0.75
